In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense
from eda.tools import seq_to_num
from preproc.filters import rnn_filter
from models.rnn import RNNData

In [ ]:
df_train = pd.read_csv('data/kaggle_train.csv', index_col=0)
X_train = seq_to_num(df_train.Sequence, pad=False, target_split=False)
df_test = pd.read_csv('data/kaggle_test.csv', index_col=0)
X_test = seq_to_num(df_test.Sequence, pad=False, target_split=False)

In [ ]:
X_combined = pd.concat([X_train, X_test], axis=0)

In [ ]:
seqlen = 10

In [ ]:
X, y = RNNData(seqlen, aug_frac=.2).transform(X_combined)

In [ ]:
X.shape

In [ ]:
def build_model(input_len):
    model = Sequential()
    model.add(LSTM(256, input_shape=(input_len, 1), return_sequences=True))
    model.add(LSTM(256))
    model.add(Dense(2000, activation='softmax'))
    model.compile('adam', 'sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
m = build_model(seqlen)

In [ ]:
m.summary()

In [ ]:
hist = m.fit(X, y, batch_size=64, epochs=3)

In [ ]:
m.save_weights(f'rnn_{seqlen}.h5')